<h1>HW4 - Attention and Transformers- Practical Q2</h1>
<h3><font color=yellow>Total Points: 100</font></h3>



<font size=+4>
Introduction
</font>
<br/>
<font size=+2>

<br/>
Part-of-speech tagging (POS tagging) is a natural language processing task where each word in a given text is assigned a specific part-of-speech category, such as noun, verb, adjective, etc. In the context of Roberta, a state-of-the-art language model based on the transformer architecture, POS tagging involves leveraging the model's pre-trained knowledge to accurately predict and label the grammatical roles of words in a sentence. Roberta's extensive training data and attention mechanisms enable it to capture subtle contextual cues, allowing for more nuanced and accurate POS tagging. By understanding the syntactic structure of a sentence, Roberta enhances the efficiency and precision of POS tagging, contributing to more sophisticated language understanding and downstream applications in natural language processing.
</font>


<font size=+2>
In this section we want to fine-tune the Roberta that you trained in previous section for POS-Tagging Task on Parsig dataset
<font/>

In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate -U

import re
import torch
import pandas as pd

import numpy as np
import transformers
import matplotlib.pyplot as plt


In [31]:
# All label that exist
# AAX mean Unknown used for CLS and other symbols
tags = {'AAX', 'B-ADJ', 'I-ADJ', 'B-ADV', 'I-ADV', 'B-N', 'I-N', 'B-V', 'I-V',\
        'B-PRONOUN', 'I-PRONOUN', 'B-NUM', 'I-NUM', 'B-DET', 'I-DET', 'B-PRE',\
        'I-PRE', 'B-POST', 'I-POST', 'B-CONJ', 'I-CONJ', 'B-JUNK', 'I-JUNK',\
        'B-MARKER', 'I-MARKER',}
# Convert lables to speceif numeric id
tag2id = {tag: id for id, tag in enumerate(sorted(list(tags)))}
id2tag = {id: tag for tag, id in tag2id.items()}

In [32]:
######################   TODO 1.1   ########################
# Load pretrained model that trained in previous sectiopn
# Load tokenizer from previous section
# Freeze base model for fine-tuning
from transformers import AutoTokenizer, AutoModelForMaskedLM
from torch import nn

tokenizer = AutoTokenizer.from_pretrained("Mahdi-Salahshour/mlm_tokenizer")
pretrained_model = AutoModelForMaskedLM.from_pretrained("Mahdi-Salahshour/mlm")


for param in pretrained_model.parameters():
    param.requires_grad = False
###################### (5 points) ##########################


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [33]:
######################   TODO 1.2  ########################
from sklearn.model_selection import train_test_split

train_df = pd.read_csv("train_df.csv")
test_df = pd.read_csv("test_df.csv")
max_length = max(max(len(seq) for seq in train_df) , max(len(seq) for seq in test_df))
def process_labels(df, tag2id):
    df['labels'] = df['labels'].str.split(',').apply(lambda x : [0] + [tag2id[i] for i in x] + [0])
    return df


train_df = process_labels(train_df, tag2id)
test_df = process_labels(test_df, tag2id)

train_df, valid_df = train_test_split(train_df, test_size=0.2, random_state=42)

###################### (5 points) ##########################


In [34]:
######################   TODO 1.3   ########################
# Complete custom data set
# Use Torcargmaxh dataloader for datasets
###################### (15 points) ##########################

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder


class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.texts = df['text'].tolist()
        self.labels = df['labels'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __getitem__(self, index):
        text = self.texts[index]
        labels = self.labels[index]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        #add padding if the length of seq is less than max length

        labels = labels[:self.max_length] + [0] * max(0, self.max_length - len(labels))

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()


        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(labels, dtype=torch.long)
        }

    def __len__(self):
        return len(self.texts)


train_dataset = CustomDataset(train_df, tokenizer, max_length)
test_dataset = CustomDataset(test_df, tokenizer, max_length)
valid_dataset = CustomDataset(valid_df, tokenizer, max_length)

batch_size = 64
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
validloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

In [45]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer

class PartOfSpeechTagger(nn.Module):
    def __init__(self, size_of_input, number_of_tags, hidden_layer):
        super(PartOfSpeechTagger, self).__init__()
        self.fully_connected_1 = nn.Linear(size_of_input, hidden_layer)
        self.batch_norm = nn.BatchNorm1d(hidden_layer)
        self.relu = nn.ReLU()
        self.fully_connected_2 = nn.Linear(hidden_layer, number_of_tags)

    def forward(self, input_tensor):
        out = self.fully_connected_1(input_tensor)
        out = self.batch_norm(out)
        out = self.relu(out)
        out = self.fully_connected_2(out)
        return out




p_h = PartOfSpeechTagger(pretrained_model.config.hidden_size, len(tags), 1024)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

roberta_model = pretrained_model.roberta

def forward(input_ids, attention_mask):
    outputs = roberta_model(input_ids=input_ids, attention_mask=attention_mask)
    last_hidden_states = outputs.last_hidden_state
    pos_output = p_h(last_hidden_states)

    return pos_output

roberta_model = roberta_model.to(device)
p_h = p_h.to(device)


In [47]:
import torch
import torch.nn.functional as F
from torch import nn
from transformers import AdamW
from tqdm import tqdm
from sklearn.metrics import accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

roberta_model.to(device)
roberta_model.train()

criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = AdamW(roberta_model.parameters(), lr=2e-5)

num_epochs = 50
num_training_steps = num_epochs * len(trainloader)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_training_steps)

for epoch in range(num_epochs):
    total_loss, total_correct, total_samples = 0, 0, 0
    tqdm_dataloader = tqdm(trainloader, desc=f"Epoch {epoch + 1}/{num_epochs}")

    for batch in tqdm_dataloader:
        input_ids, attention_mask, labels = [item.to(device) for item in (batch['input_ids'], batch['attention_mask'], batch['labels'])]
        optimizer.zero_grad()
        outputs = roberta_model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        loss = criterion(logits.view(-1, roberta_model.config.num_labels), labels.view(-1))
        total_loss += loss.item()
        _, predicted_labels = torch.max(logits, 2)
        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.numel()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

    print(f"Epoch {epoch + 1}/{num_epochs} - Average Loss: {total_loss / len(trainloader)}, Accuracy: {total_correct / total_samples * 100:.2f}%")

    roberta_model.eval()
    total_valid_loss, total_valid_correct, total_valid_samples = 0, 0, 0

    with torch.no_grad():
        tqdm_valid_dataloader = tqdm(validloader, desc=f"Epoch {epoch + 1}/{num_epochs} - Validation")

        for valid_batch in tqdm_valid_dataloader:
            input_ids_valid, attention_mask_valid, labels_valid = [item.to(device) for item in (valid_batch['input_ids'], valid_batch['attention_mask'], valid_batch['labels'])]
            valid_outputs = roberta_model(input_ids_valid, attention_mask=attention_mask_valid, labels=labels_valid)
            valid_logits = valid_outputs.logits
            valid_loss = F.cross_entropy(valid_logits.view(-1, roberta_model.config.num_labels), labels_valid.view(-1), ignore_index=tokenizer.pad_token_id)
            total_valid_loss += valid_loss.item()
            _, predicted_labels_valid = torch.max(valid_logits, 2)
            total_valid_correct += (predicted_labels_valid == labels_valid).sum().item()
            total_valid_samples += labels_valid.numel()

    print(f"Epoch {epoch + 1}/{num_epochs} - Validation Loss: {total_valid_loss / len(validloader)}, Validation Accuracy: {total_valid_correct / total_valid_samples * 100:.2f}%")

    roberta_model.train()



Epoch 1/50: 100%|██████████| 41/41 [00:04<00:00,  8.96it/s]


Epoch 1/50 - Average Loss: 2.2528280165137313, Accuracy: 36.82%


Epoch 1/50 - Validation: 100%|██████████| 11/11 [00:00<00:00, 27.46it/s]


Epoch 1/50 - Validation Loss: 1.661430142142556, Validation Accuracy: 50.69%


Epoch 2/50: 100%|██████████| 41/41 [00:04<00:00,  9.76it/s]


...

Epoch 49/50 - Validation Loss: 0.7115216065536846, Validation Accuracy: 82.75%


Epoch 50/50: 100%|██████████| 41/41 [00:05<00:00,  7.83it/s]


Epoch 50/50 - Average Loss: 0.10962375135319989, Accuracy: 94.59%


Epoch 50/50 - Validation: 100%|██████████| 11/11 [00:00<00:00, 20.37it/s]

Epoch 50/50 - Validation Loss: 0.6903644339604811, Validation Accuracy: 82.75%


In [48]:
######################   TODO 2.2   ########################
# Report best model accuarcy on validation dataset
# Accuracy below 85% will not be graded
# accuracy_valid
roberta_model.eval()
total_valid_correct = 0
total_valid_samples = 0
total_valid_loss = 0

with torch.no_grad():
    t = tqdm(testloader, desc=f"Epoch {epoch + 1}/{num_epochs} - Validation")

    for valid_batch in t:
        input_ids_valid, attention_mask_valid, labels_valid = valid_batch['input_ids'], valid_batch['attention_mask'], valid_batch['labels']

        input_ids_valid, attention_mask_valid, labels_valid = input_ids_valid.to(device), attention_mask_valid.to(device), labels_valid.to(device)

        valid_outputs = roberta_model(input_ids_valid, attention_mask=attention_mask_valid, labels=labels_valid)
        valid_logits = valid_outputs.logits

        valid_loss = F.cross_entropy(valid_logits.view(-1, roberta_model.config.num_labels), labels_valid.view(-1), ignore_index=tokenizer.pad_token_id)

        total_valid_loss += valid_loss.item()

        _, predicted_labels_valid = torch.max(valid_logits, 2)
        correct_valid = (predicted_labels_valid == labels_valid).sum().item()
        total_valid_correct += correct_valid
        total_valid_samples += labels_valid.numel()

average_valid_loss = total_valid_loss / len(validloader)
accuracy_valid = total_valid_correct / total_valid_samples
print(f"Epoch {epoch + 1}/{num_epochs} - Validation Loss: {average_valid_loss}, Validation Accuracy: {accuracy_valid * 100:.2f}%")

###################### (5 points) ##########################

Epoch 50/50 - Validation: 100%|██████████| 7/7 [00:00<00:00, 19.53it/s]

Epoch 50/50 - Validation Loss: 0.4731152057647705, Validation Accuracy: 82.27%


In [51]:
######################   TODO 2.1   ########################
# Now implement it with huggingface trainer

from transformers import AdamW, Trainer, TrainingArguments
import torch.nn as nn

loss_function = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

class MyTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs, labels=labels)
        logits = outputs.logits
        loss = loss_function(logits.view(-1, logits.shape[-1]), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

training_arguments = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    metric_for_best_model="eval_loss",
)

my_trainer = MyTrainer(
    model=roberta_model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

my_trainer.train()

###################### (40 points) ##########################

Step,Training Loss,Validation Loss
50,0.042200,1.024700
100,0.022000,1.077994
150,0.022500,1.144989
200,0.025400,1.156328
250,0.025300,1.114691
300,0.018400,1.128590
350,0.012100,1.146529
400,0.008500,1.145519


TrainOutput(global_step=410, training_loss=0.021715456974215622, metrics={'train_runtime': 55.6664, 'train_samples_per_second': 470.302, 'train_steps_per_second': 7.365, 'total_flos': 80181698058000.0, 'train_loss': 0.021715456974215622, 'epoch': 10.0})